How to set up selenium and client?

In [ ]:
var importer = require('../Core');

var client, results, getCredentials, loginGoogle, getAllXPath;
var getClient = () => {
    return importer.interpretAll([
        'webdriver client',
        'decrypt password',
        'log in Google',
        'get all elements'
    ])
    .then(r => {
        results = r;
        return r[0].runInNewContext();
    })
    .then(r => {
        client = r();
        getCredentials = results[1].runInNewContext({client});
        loginGoogle = results[2].runInNewContext({client, getCredentials});
        getAllXPath = results[3].runInNewContext({client});
        return client;
    });
};
module.exports = getClient;
getClient;



In [ ]:
$$.async();
getClient()
    .then(c => $$.sendResult(c))
    .catch(e => $$.sendError(e));


How to download google timeline data using webdriver?

In [ ]:
var getGoogleTimeline = () => {
    console.log('Google: Logging timeline history');
    return client
    .url('https://www.google.com/maps/timeline')
    .loginGoogle()
    .waitUntil(() => client.getUrl()
               .then(url => url.indexOf('timeline') > -1, 20000, ''))
    .catch(function (e) {
        console.log(e);
    })
    .pause(3000)
    .click('button[jsaction="header.select-today"]')
    .pause(2000);
}
if (typeof client.getGoogleTimeline == 'undefined') {
    client.addCommand('getGoogleTimeline', getGoogleTimeline);
}
module.exports = getGoogleTimeline;
getGoogleTimeline;


In [ ]:
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var PROJECT_PATH = '/Users/briancullinan/Timeline';

var readAllPages = () => {
    // TODO: add stop conditions
    var oneDay = 24*60*60*1000; // hours*minutes*seconds*milliseconds
    var firstDate = new Date(2016,8,21);
    var secondDate = new Date();
    var days = Math.round(
        Math.abs((firstDate.getTime() - secondDate.getTime())/(oneDay)));
    dayKey = '';
    var promises = [];
    for (var c = 0; c < days; c++) {
        var result;
        promises[promises.length] = (resolve) => readTimelinePage()
            .then(r => result = r)
            .click('.previous-date-range-button')
            .pause(2000)
            .then(() => {
                if(typeof(result) == 'undefined') {
                    resolve();
                }
                console.log(result);
                fs.writeFileSync(
                    PROJECT_PATH + '/timeline-' + result[0].timeline + '_.json',
                    JSON.stringify(result, null, 4))
                resolve(result);
            })
    }
    return promises;
}
if (typeof client.getGoogleTimeline == 'undefined') {
    client.addCommand('readAllPages', readAllPages);
}
module.exports = readAllPages;
readAllPages;


In [ ]:
var chrono = require('chrono-node');

var months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'];
var readTimelinePage = () => {
    return client.getAllXPath({
        day: '//*[contains(@class,"timeline-subtitle")]//text()',
        items: [
            '//*[contains(@class,"timeline-item")]/parent::*/*[@jsinstance]',
            {
                duration: './/*[contains(@class, "duration-text")]//text()',
                data: './/*[contains(@class, "timeline-item")]/@data-segment-key',
                title: './/*[contains(@class, "timeline-item-title-content")][.//i]/*[not(self::i)]//text()|.//*[contains(@class, "timeline-item-title-content")][not(.//i)]//text()',
                location: './/*[contains(@class, "timeline-item-text")][not(contains(@class,"add-child"))][.//a]/a//*[not(self::i)]//text()|.//*[contains(@class, "timeline-item-text")][not(contains(@class,"add-child"))][not(.//a)]/text()'
            }
        ]
    })
    .then(r => {
        console.log(r.day);
        if(typeof r.day != 'string') {
            return;
        }
        var currDate = chrono.parseDate(r.day);
        var newKey = currDate.getDate() 
            + months[currDate.getMonth()] 
            + (currDate.getFullYear() + '').substr(2, 2);
        if (newKey != dayKey) {
            dayKey = newKey;
        }
        
        return r.items.map(i => {
            var timelineData = (i.data + '').split(':');
            var start, end, length;
            if(timelineData[0]) {
                start = new Date(parseFloat(timelineData[1]));
                end = new Date(parseFloat(timelineData[2]));
                length = end.getTime() - start.getTime();
            } else {
                start = chrono.parseDate(r.day + ' ' + i.duration.join('').trim().split('-')[0]);
                end = chrono.parseDate(r.day + ' ' + i.duration.join('').trim().split('-')[1]);
                length = 0;
            }
            return ({
                type: 'timeline',
                timeline: dayKey,
                name: i.title,
                location: i.location,
                time: start,
                length: isNaN(length) ? 0 : length
            })
        })
    })
    .catch(e => console.log(e))
};
if (typeof client.readTimelinePage == 'undefined') {
    client.addCommand('readTimelinePage', readTimelinePage);
}
module.exports = readTimelinePage;
readTimelinePage;


In [ ]:
var fs = require('fs');
var importer = require('../Core');

$$.async()
client.getGoogleTimeline()
    .then((r) => importer.runAllPromises(readAllPages()))
    .catch(e => $$.sendError(e));



In [45]:
var _ = require('underscore');
var importer = require('../Core');
var months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'];

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var PROJECT_PATH = '/Users/briancullinan/Timeline';

function toRadians (angle) {
  return angle * (Math.PI / 180);
}

var straightDistance = (lat1, lon1, lat2, lon2) => {
    var R = 6371e3; // metres
    var φ1 = toRadians(lat1);
    var φ2 = toRadians(lat2);
    var Δφ = toRadians(lat2-lat1);
    var Δλ = toRadians(lon2-lon1);

    var a = Math.sin(Δφ/2) * Math.sin(Δφ/2) +
            Math.cos(φ1) * Math.cos(φ2) *
            Math.sin(Δλ/2) * Math.sin(Δλ/2);
    var c = 2 * Math.atan2(Math.sqrt(a), Math.sqrt(1-a));

    var d = R * c;
    return d;
}

var locationCache = {};
var listEvents, placesNearby, getOauthClient, ISODateString;
var dayKey, count = 0, options = {};
$$.async();
importer.streamJson('/Users/briancullinan/Downloads/Takeout 7/Location History/Location History.json', [true, {emitPath: true}], (match) => {
    if(match.path[0] === 'locations') {
        var currDate = new Date(parseInt(match.value.timestampMs));
        var newKey = currDate.getDate() + months[currDate.getMonth()]
            + (currDate.getFullYear() + '').substr(2, 2);
        if (newKey != dayKey) {
            dayKey = newKey;
        }
        var newRow = Object.assign({
                time: currDate,
                type: 'location',
                location: dayKey,
        }, match.value);
        var cache = locationCache[newRow.location];
        if(typeof cache === 'undefined') {
            cache = locationCache[newRow.location] = [];
        }
        cache[cache.length] = newRow;
    }
})
.then(r => importer.interpretAll([
    'places nearby google maps',
    'import google calendar api',
    'list events',
    'date iso',
]))
.then(r => {
    placesNearby = r[0].runInNewContext();
    getOauthClient = r[1].runInNewContext();
    listEvents = r[2].runInNewContext();
    ISODateString = r[3].runInNewContext();
    return getOauthClient(options);
})
.then(() => importer.runAllPromises(Object.keys(locationCache).slice(10, 11)
    .map(date => resolve => {
    if(!fs.existsSync(PROJECT_PATH + '/timeline-' + date + '_.json')) {
        return resolve();
    }
    var timelineData = JSON.parse(
        fs.readFileSync(PROJECT_PATH + '/timeline-' + date + '_.json')
        .toString());
    var destinations = [];
    for(const d of timelineData) {
        if((d.name + '') == '' || (d.name + '').indexOf('Driving') > -1) {
            continue;
        }
        const nearest = _.sortBy(
            locationCache[date],
            l => Math.abs(new Date(l.time).getTime() - new Date(d.time).getTime()))
            .slice(0, 3);
        // make sure it isn't off by much
        const averageLat = nearest.map(n => n.latitudeE7)
            .reduce((a, b) => a + b, 0) / nearest.length;
        const averageLon = nearest.map(n => n.longitudeE7)
            .reduce((a, b) => a + b, 0) / nearest.length;
        if(nearest.filter(n => straightDistance(
                n.latitudeE7 / 10000000,
                n.longitudeE7 / 10000000,
                averageLat / 10000000,
                averageLon / 10000000
            ) > 500).length > 0) {
            destinations.push(Object.assign(d, {
                averageLat: averageLat / 10000000,
                averageLon: averageLon / 10000000,
                locations: nearest.map(n => Object.assign(n, {
                    averageLat: n.latitudeE7 - averageLat,
                    averageLon: n.longitudeE7 - averageLon,
                    averageDist: straightDistance(
                        n.latitudeE7 / 10000000,
                        n.longitudeE7 / 10000000,
                        averageLat / 10000000,
                        averageLon / 10000000
                    )
                }))
            }));
        }
        destinations.push(Object.assign(d, {
            averageLat: averageLat / 10000000,
            averageLon: averageLon / 10000000
        }));
    }

    //return resolve(destinations);
    var locations;
    return importer.runAllPromises(destinations.map(d => resolve => {
        var lat_long = {lat: d.averageLat, lng: d.averageLon};
        return placesNearby((d.location + ', ' + d.name), lat_long)
            .then(r => Object.assign(d, r[0]))
            .then(r => setTimeout(() => resolve(r), 100))
            .catch(e => setTimeout(() => resolve(e), 100));
    }))
    .then(r => {
        locations = r;
        const start = new Date(r[0].time);
        start.setHours(0, 0, 0);
        const end = new Date(r[0].time);
        end.setHours(23, 59, 0);
        return listEvents({
            auth: options.auth,
            timeMin: start.toString(),
            timeMax: end.toString()
        });
    })
    .then(events => {
        console.log(events);
        const unmatched = [], matches = [];
        for(const l of locations) {
            const matching = events.filter(e => {
                return ((l.name + '').indexOf('Gainey') > -1
                    && e.event.summary == 'Drive to work')
                || ((l.name + '').indexOf('Swiftpage') > -1
                    && e.event.summary == 'Drive to work')
                || ((l.name + '').indexOf('6934') > -1
                    && e.event.summary == 'Drive home')
                || (typeof e.event.location !== 'undefined'
                    && ((e.event.location + '').indexOf(l.location + '') > -1
                    || (e.event.location + '').indexOf(l.name + '') > -1))
            });
            if(matching.length == 0) {
                unmatched.push(l);
            } else {
                matches.push(Object.assign(l, matching[0]));
            }
        }
        console.log(unmatched);
        console.log(matches);
    })
    .then(r => resolve(r))
    .catch(e => resolve(e))
})))
.then(r => $$.sendResult(r))
.catch(e => $$.sendError(e))



Already imported
[ 'import notebook.ipynb',
  'interpret all notebooks.ipynb',
  'google authorize.ipynb' ]


[ { start: 2017-09-20T15:00:00.000Z,
    event: 
     { kind: 'calendar#event',
       etag: '"3011842185660000"',
       id: '4k1bf4d49qpu56kcgn459arli3',
       status: 'confirmed',
       htmlLink: 'https://www.google.com/calendar/event?eid=NGsxYmY0ZDQ5cXB1NTZrY2duNDU5YXJsaTMgbWVnYW1pbmRicmlhbkBt',
       created: '2017-09-20T15:24:06.000Z',
       updated: '2017-09-20T15:24:52.830Z',
       summary: 'Drive to work',
       description: 'Cried on the way to work.  Thinking about Midnight Special and how we lack sympathy for others.  Is Midnight Special wasn\'t fiction what does that mean for our world?\n',
       creator: [Object],
       organizer: [Object],
       start: [Object],
       end: [Object],
       iCalUID: '4k1bf4d49qpu56kcgn459arli3@google.com',
       sequence: 0,
       reminders: [Object] } },
  { start: 2017-09-20T16:00:00.000Z,
    event: 
     { kind: 'calendar#event',
       etag: '"3011716497358000"',
       id: '349f31alt6mlb60vmglsgvvvsr',
       status: 'co

[ undefined ]